In [ ]:
import mcmc_utils

In [ ]:
import surp
import vice
import numpy as np

import matplotlib.pyplot as plt

from surp.gce_math import Z_to_MH, MH_to_Z
from surp.yields import calc_y

In [ ]:
from singlezone import run_singlezone

In [ ]:
import arya

In [ ]:
surp.set_yields()

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
from surp.agb_interpolator import interpolator as agb_interpolator

## Single zone approximation

In [ ]:
def agb_interpolator(ele, study, **kwargs):
    return surp.agb_interpolator.interpolator(ele, study=study, **kwargs)

In [ ]:
import warnings

the final carbon abundance is
$$
Z_{\rm C}(t_{\rm end}) = \frac{y_{\rm C}}{1+\eta - r - \tau_{\star} / \tau_{\rm sfh}}
$$

In [ ]:
def calc_y_sz(ele="c", ignore_warnings=True):
    if ignore_warnings:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            sz, h = run_singlezone()
    
    Z_end = h[f"z({ele})"].iloc[-1]
    tau_star = 2.5
    η = 0.5
    r = 0.4
    tau_sfh = 14
    
    y0 = Z_end * (1 + η - r - tau_star / tau_sfh)
    
    return y0

In [ ]:
vice.yields.ccsne.settings["fe"] + vice.yields.sneia.settings["fe"]

In [ ]:
calc_y_sz("fe")

In [ ]:
vice.yields.ccsne.settings["c"] = 1e-4
vice.yields.agb.settings["c"] = agb_interpolator("c", study="cristallo11", prefactor=0)

calc_y_sz("c")

In [ ]:
vice.yields.ccsne.settings["c"] = 0

for model in surp.AGB_MODELS:
    vice.yields.agb.settings["c"] = agb_interpolator("c", study=model)
    y = calc_y_sz("c")

    print(model, "\t", f"{y:0.3e}")


In [ ]:
for m_factor in [0.5, 0.8, 1, 1.3, 2]:
    vice.yields.agb.settings["c"] = agb_interpolator("c", study="cristallo11", mass_factor=m_factor)
    y = calc_y_sz("c")

    print(m_factor, "\t", f"{y:0.3e}")

# MCMC Fits

# Prior models

In [ ]:
lin_model = mcmc_utils.Linear(beta0=[3e-4, 3e-4], beta1=[0, 6e-4], log_sigma=[-20, 1])
exp_model = mcmc_utils.Exponential(beta0=[3e-4, 3e-3], beta1=[0,2e-2], log_sigma=[-20, 1])
quad_model = mcmc_utils.Quadratic(beta0=[3e-4, 3e-4], beta1=[0, 3e-4], beta2=[0, 3e-4], log_sigma=[-20, 1])

In [ ]:
N = 3_000
discard = 300

In [ ]:
mcmc_utils.plot_prior(lin_model)

In [ ]:
mcmc_utils.plot_prior(exp_model, xlims=(-2, 0.3))

In [ ]:
mcmc_utils.plot_prior(quad_model)

# Fit to the yield tables

In [ ]:
def get_yield_table(study, **kwargs):
    y1, m1, z1 = vice.yields.agb.grid("c", study=study)
    mh1 = Z_to_MH(z1)
    vice.yields.agb.settings["c"] = agb_interpolator("c", study=study, **kwargs)
    y_c_agb1 = np.array([calc_y(z, kind="agb") for z in z1])

    return mh1, y_c_agb1, None, None

In [ ]:
obs = get_yield_table("cristallo11")

In [ ]:
plt.scatter(obs[0], obs[1])

In [ ]:
def lin_ls(x, a, b):
    return a + b*x

def quad_ls(x, a, b, c):
    return a + b*x + c*x**2

def exp_ls(x, a, b, c):
    return a + b*np.exp(c*x)


In [ ]:
def fit_ls(obs, model, p0=None, color=arya.COLORS[1], **kwargs):
    popt, covt = curve_fit(model, obs[0], obs[1], p0=None)

    for i in range(len(popt)):
        print(f"{popt[i]:8.3e}  ±  {np.sqrt(covt[i,i]):8.3e}")

    x = np.linspace(np.min(obs[0]), np.max(obs[0]), 1000)
    y = model(x, *popt)
    plt.plot(x, y, color=color, **kwargs)

    return popt

### Mass factor

In [ ]:
y_tot = (2.04 * 0.382 + 2.01)

In [ ]:
f_agb = (2.04 * 0.382) / y_tot

In [ ]:
alpha_0 = 2.04
a_0 = 0.382 

In [ ]:
2.3 / 2.04, 1.795 / 2.04

In [ ]:
for mass_factor in [0.5, 0.7, 1,1.3, 2]:
    print("mass factor = ", mass_factor)
    obs = get_yield_table("cristallo11", mass_factor=mass_factor)
    a, b = fit_ls(obs, lin_ls, color=None)
    plt.scatter(obs[0], obs[1])

    alpha_exp = alpha_0 * (a) / 0.382
    print("alpha = ", alpha_exp)
    print()

In [ ]:
obs = get_yield_table("cristallo11")

### Normal investivation

In [ ]:
samples = mcmc_utils.run_mcmc(lin_model, obs, N=N, discard=discard)

In [ ]:
mcmc_utils.plot_posterior(lin_model, obs, samples)
fit_ls(obs, lin_ls)

In [ ]:
samples = mcmc_utils.run_mcmc(quad_model, obs, N=N, discard=discard)

In [ ]:
mcmc_utils.plot_posterior(quad_model, obs, samples)
plt.ylim(-0.3e-3, 2e-3)
fit_ls(obs, quad_ls)

In [ ]:
samples = mcmc_utils.run_mcmc(exp_model, obs, N=N, discard=discard)

In [ ]:
mcmc_utils.plot_posterior(exp_model, obs, samples)
fit_ls(obs, exp_ls)
plt.ylim(-1e-3, 2e-3)

# V13

In [ ]:
obs = get_yield_table("ventura13")

plt.scatter(obs[0], obs[1])
obs = list(obs)
obs[0] = obs[0][1:]
obs[1] = obs[1][1:]
plt.scatter(obs[0], obs[1])


In [ ]:
samples = mcmc_utils.run_mcmc(lin_model, obs, N=N, discard=discard)

In [ ]:
mcmc_utils.plot_posterior(lin_model, obs, samples)
fit_ls(obs, lin_ls)


In [ ]:
samples = mcmc_utils.run_mcmc(quad_model, obs, N=N, discard=discard)

In [ ]:
mcmc_utils.plot_posterior(quad_model, obs, samples)
fit_ls(obs, quad_ls)


In [ ]:
samples = mcmc_utils.run_mcmc(exp_model, obs, N=N, discard=discard)

In [ ]:
mcmc_utils.plot_posterior(exp_model, obs, samples)
fit_ls(obs, exp_ls)

plt.ylim(-1e-3, 2e-3)

# K16

In [ ]:
obs = get_yield_table("karakas16")

In [ ]:
plt.scatter(obs[0], obs[1])

In [ ]:
samples = mcmc_utils.run_mcmc(lin_model, obs, N=N, discard=discard)

In [ ]:
mcmc_utils.plot_posterior(lin_model, obs, samples)
fit_ls(obs, lin_ls)


In [ ]:
samples = mcmc_utils.run_mcmc(quad_model, obs, N=N, discard=discard)

In [ ]:
mcmc_utils.plot_posterior(quad_model, obs, samples)
fit_ls(obs, quad_ls)


In [ ]:
samples = mcmc_utils.run_mcmc(exp_model, obs, N=N, discard=discard)

In [ ]:
mcmc_utils.plot_posterior(exp_model, obs, samples)
fit_ls(obs, exp_ls)

plt.ylim(-1e-3, 2e-3)

# P16

In [ ]:
obs = get_yield_table("battino19")

In [ ]:
plt.scatter(obs[0], obs[1])

In [ ]:
samples = mcmc_utils.run_mcmc(lin_model, obs, N=N, discard=discard)

In [ ]:
mcmc_utils.plot_posterior(lin_model, obs, samples)
fit_ls(obs, lin_ls)


In [ ]:
samples = mcmc_utils.run_mcmc(quad_model, obs, N=N, discard=discard)

In [ ]:
mcmc_utils.plot_posterior(quad_model, obs, samples)
fit_ls(obs, quad_ls)

plt.ylim(-0.3e-3, 3e-3)

In [ ]:
samples = mcmc_utils.run_mcmc(exp_model, obs, N=N, discard=1000)

In [ ]:
mcmc_utils.plot_posterior(exp_model, obs, samples)
#fit_ls(obs, exp_ls, p0=[3.5e-3, -2.6e-3, -1.79e1])

plt.ylim(-0.3e-3, 3e-3)

In [ ]:
obs = get_yield_table("pignatari16")
#obs = (obs[0][1:], obs[1][1:], None, None)

In [ ]:
plt.scatter(obs[0], obs[1])

In [ ]:
samples = mcmc_utils.run_mcmc(lin_model, obs, N=N, discard=discard)

In [ ]:
mcmc_utils.plot_posterior(lin_model, obs, samples)
fit_ls(obs, lin_ls)


# M factor

In [ ]:
obs0 = get_yield_table("cristallo11")

In [ ]:
obs = get_yield_table("cristallo11", mass_factor = 0.5)

In [ ]:
plt.scatter(obs[0], obs[1])
plt.scatter(obs0[0], obs0[1])

In [ ]:
samples = mcmc_utils.run_mcmc(lin_model, obs, N=N, discard=discard)

In [ ]:
mcmc_utils.plot_posterior(lin_model, obs, samples)

In [ ]:
samples = mcmc_utils.run_mcmc(quad_model, obs, N=N, discard=discard)

In [ ]:
mcmc_utils.plot_posterior(quad_model, obs, samples)

In [ ]:
samples = mcmc_utils.run_mcmc(exp_model, obs, N=N, discard=discard)

In [ ]:
mcmc_utils.plot_posterior(exp_model, obs, samples)
plt.ylim(-1e-3, 2e-3)

In [ ]:
get_yield_table